In [1]:
pip install tensorflow matplotlib

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-03-11 13:51:12.867519: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-11 13:51:13.039010: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Defining the dataset path
data_dir = "ML Dataset"

#Define image size and batch size
IMG_size = 128
BATCH_size = 32

#Create ImageDataGenerator for Training and Validation
#Data Augmentation
train_datagen = ImageDataGenerator(
    rescale = 1./255,           # Normalizing pixel values
    rotation_range = 40,        # Rotating images
    width_shift_range = 0.3,    # Shifting image width
    height_shift_range = 0.3,   # Shifting image height
    shear_range = 0.3,          # Shear transformation
    zoom_range = 0.3,           # Zoom in and out
    horizontal_flip = True,     # Fliping images horizontally 
    brightness_range=[0.7, 1.3],
    fill_mode='nearest',
    validation_split = 0.2      # Splitting 20% data into validation
)

# Loading training data
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size = (IMG_size, IMG_size),
    batch_size = BATCH_size,
    class_mode = "binary",        # using binary class mode
    subset = "training"
)

# Loading validation data
val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size = (IMG_size, IMG_size),
    batch_size = BATCH_size,
    class_mode = "binary",
    subset = "validation"
)

Found 192 images belonging to 2 classes.
Found 48 images belonging to 2 classes.


In [14]:
# Checking Dataset details

print("Class Labels :", train_generator.class_indices)
print(f"Total Training Samples : {train_generator.samples}")
print(f"Total Validation Samples : {val_generator.samples}")

Class Labels : {'Mask': 0, 'No Mask': 1}
Total Training Samples : 192
Total Validation Samples : 48


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2

# Initializing the CNN Model
model =Sequential()

# Adding 1st Conolutional Layer
model.add(Conv2D(32,(3,3), activation = 'relu', input_shape=(IMG_size, IMG_size,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Adding 2nd Convolutional Layer
model.add(Conv2D(64,(3,3), activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Adding 3rd Convolutional Layer
model.add(Conv2D(128,(3,3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Adding the Flatten Layer
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(512, activation = 'relu', kernel_regularizer=l2(0.01)))

# Adding Dropout Layer
model.add(Dropout(0.4))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

#Output Layer
model.add(Dense(1, activation = 'sigmoid'))

# Compiling the Model
optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

# Summary of the Mode
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 batch_normalization_3 (Bat  (None, 126, 126, 32)      128       
 chNormalization)                                                
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 61, 61, 64)        256       
 chNormalization)                                                
                                                      

In [16]:
# Training the Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Reduce learning rate when loss stops improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

history = model.fit(
    train_generator,                                         # Training data
    steps_per_epoch = train_generator.samples // BATCH_size, # No of training batches
    epochs = 30,
    validation_data = val_generator,                         # Validation data
    validation_steps = val_generator.samples // BATCH_size,   # No of validation batches
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/30
6/6 [==============================] - 19s 3s/step - loss: 12.1051 - accuracy: 0.5052 - val_loss: 11.3305 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 2/30
6/6 [==============================] - 16s 3s/step - loss: 11.7682 - accuracy: 0.5521 - val_loss: 11.0786 - val_accuracy: 0.7500 - lr: 0.0100
Epoch 3/30
6/6 [==============================] - 16s 3s/step - loss: 11.7084 - accuracy: 0.6094 - val_loss: 11.6000 - val_accuracy: 0.4375 - lr: 0.0100
Epoch 4/30
6/6 [==============================] - 15s 3s/step - loss: 11.7039 - accuracy: 0.5885 - val_loss: 11.3025 - val_accuracy: 0.5625 - lr: 0.0100
Epoch 5/30
6/6 [==============================] - 16s 3s/step - loss: 11.8581 - accuracy: 0.6146 - val_loss: 10.9211 - val_accuracy: 0.4062 - lr: 0.0100
Epoch 6/30
6/6 [==============================] - 17s 3s/step - loss: 11.2285 - accuracy: 0.6302 - val_loss: 10.8257 - val_accuracy: 0.5938 - lr: 0.0100
Epoch 7/30
6/6 [==============================] - 16s 3s/step - loss: 10.8104 - ac

In [17]:
# Evaluating the model on the validation set

train_loss, train_acc = model.evaluate(train_generator)
print(f"Train Loss : {train_loss}, Train Accuracy : {train_acc}")

val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Loss : {val_loss}, Validation Accuracy : {val_acc}")

6/6 [==============================] - 12s 2s/step - loss: 8.9369 - accuracy: 0.5573
Train Loss : 8.936928749084473, Train Accuracy : 0.5572916865348816
2/2 [==============================] - 4s 1s/step - loss: 8.9412 - accuracy: 0.5417
Validation Loss : 8.941235542297363, Validation Accuracy : 0.5416666865348816


In [18]:
# Saving the Model
model.save("mask_detection_model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [19]:
from tensorflow.keras.models import load_model

# Loading the saved model
model = load_model("mask_detection_model.h5")

In [20]:
# Testing with an Image

import numpy as np
from tensorflow.keras.preprocessing import image

# Loading the test image
img_path = "Image3.jpg"
img = image.load_img(img_path, target_size = (128,128))

# Converting the image to an array
img_array = image.img_to_array(img)

# Adding a batch dimension
img_array = np.expand_dims(img_array, axis=0)

#Normalizing the pixel values of the test image
img_array = img_array/255.0

In [21]:
# Making the Prediction

prediction = model.predict(img_array)

print(prediction)
# Output the result
if prediction[0][0] > 0.5:
    print("No Mask Detected")
else:
    print("Mask Detected")

1/1 [==============================] - 0s 74ms/step
[[0.5276127]]
No Mask Detected
